## Intro
The overall processing steps:
* read train and test file. change date type. e.g. time related columns to datetime type
* create train data set:
    * Explore the max order pay time.
    * Use the last year records as label. whether the customers purchase or not purchase
    * Use the records before the last year as train data
* Feature engineering
    * Extract customer last action in goods, order related features
    * mean. max. std purchase good's count and orders
    * Time related columns, we need to extract year, month, day, workday,time interval between the last and earlist order(rfm is better)
* create baseline to go through
     * for each record in train data set, label those purchase in Aug as 1 and the others as 0
     * split train test data set
     * use the whole data set to predict to get final results

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
%matplotlib inline


## Read data files

In [2]:
path ='./Data/UserPurchasePrediction/'
df = pd.read_csv(path+'train.csv')
result = pd.read_csv(path + 'submission.csv')

In [3]:
print(df.columns)
print(result.head())
print(df.dtypes)
df[['order_pay_time','goods_list_time','goods_delist_time']] =\
df[['order_pay_time','goods_list_time','goods_delist_time']].apply(pd.to_datetime)

print(df.dtypes)
df.head()

Index(['order_detail_id', 'order_id', 'order_total_num', 'order_amount',
       'order_total_payment', 'order_total_discount', 'order_pay_time',
       'order_status', 'order_count', 'is_customer_rate',
       'order_detail_status', 'order_detail_goods_num', 'order_detail_amount',
       'order_detail_payment', 'order_detail_discount', 'customer_province',
       'customer_city', 'member_id', 'customer_id', 'customer_gender',
       'member_status', 'is_member_actived', 'goods_id', 'goods_class_id',
       'goods_price', 'goods_status', 'goods_has_discount', 'goods_list_time',
       'goods_delist_time'],
      dtype='object')
   customer_id  result
0      1000000     0.0
1      1000014     0.0
2      1000034     0.0
3      1000046     0.0
4      1000048     0.0
order_detail_id             int64
order_id                    int64
order_total_num           float64
order_amount              float64
order_total_payment       float64
order_total_discount      float64
order_pay_time         

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,customer_gender,member_status,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time
0,1000000,1000000,1.0,239.9,96.9,0.0,2012-11-01 00:10:56,6,1.0,0.0,...,NaN,NaN,NaN,998,998,54.909289,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
1,1001530,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1953,1953,45.961352,0.0,1.0,2013-08-28 17:27:50,2013-09-01 00:38:17
2,1001531,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1083,1083,53.035439,1.0,0.0,2014-10-29 18:21:05,2014-11-05 18:21:05
3,1001532,1001328,3.0,180.0,89.7,0.0,2013-08-31 22:06:35,6,1.0,0.0,...,NaN,NaN,NaN,1013,1013,46.046917,1.0,1.0,2014-10-25 11:00:00,2014-11-01 11:00:00
4,1001533,1001329,1.0,159.9,65.9,0.0,2013-08-31 21:33:36,6,1.0,0.0,...,NaN,NaN,NaN,1628,1628,50.722161,1.0,0.0,2014-10-23 15:35:33,2014-10-30 15:35:33


## Create Train data set

In [4]:
# This is to predict the user purchase action, so we need to use historical data to predict
# the  user behavior

In [5]:
print(max(df['order_pay_time']))

2013-08-31 23:59:59


In [6]:
# Using data less equal to July create train data
print(df.customer_id.nunique())
print(df.shape)
df[(df['order_pay_time']<= '2013-08-31 23:59:59') & (df['order_pay_time']> '2013-08-01 00:00:01')].shape

1585986
(2306871, 29)


(226168, 29)

In [157]:
train_raw =df[df['order_pay_time']<= '2013-07-31 23:59:59'] 
train_raw.sort_values(by='order_pay_time', inplace =True)
# customer who purchase on august
label_raw = df[df['order_pay_time']> '2013-07-31 23:59:59']['customer_id'].dropna()
print(train_raw.shape)
label_raw.shape

(2080703, 29)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(226168,)

## Feature Engineering
### label encodeing the province and city info

In [158]:
# explore the missing city and province records
print(train_raw[train_raw['customer_city'].isnull()].order_pay_time.describe())
print(f"There are total {len(set(train_raw[train_raw['customer_city'].isnull()].customer_id))} ")
print(f"there are {len(set(train_raw[train_raw['customer_city'].isnull()].customer_id).intersection(set(label_raw)))} in label set")

# they are limited very earlier records but having purchase action in Aug. 
# so we will use value as 'missing' to replace the na value
train_raw_copy = train_raw.copy(deep = True)
train_raw['customer_province'].fillna('missing', inplace=True)
train_raw['customer_city'].fillna('missing', inplace=True)
df['customer_province'].fillna('missing', inplace=True)
df['customer_city'].fillna('missing', inplace=True)

count       0
unique      0
top       NaN
freq      NaN
Name: order_pay_time, dtype: object
There are total 0 
there are 0 in label set


C:\Anaconda3\lib\site-packages\pandas\core\series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [159]:
cols = ['customer_province', 'customer_city']
for col in cols:
    label_encoder= LabelEncoder()
    train_raw[col+'_encoded'] = label_encoder.fit_transform(train_raw[col])
    df[col+'_encoded'] = label_encoder.transform(df[col])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [85]:
train_raw.head()

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time,customer_province_encoded,customer_city_encoded
2021321,3349677,2975399,1.0,199.0,66.9,0.0,2012-11-01 00:00:07,1,1.0,0.0,...,1.0,1063,1063,53.046846,1.0,0.0,2014-10-24 12:00:33,2014-10-31 12:00:33,13,135
898720,2042828,1890070,1.0,239.9,96.9,0.0,2012-11-01 00:00:14,6,1.0,0.0,...,NaN,1353,1353,54.865584,1.0,0.0,2014-10-24 09:52:16,2014-10-31 09:52:16,16,280
1064326,2236278,2054851,1.0,199.0,66.9,0.0,2012-11-01 00:00:21,6,1.0,0.0,...,NaN,1063,1063,53.029011,1.0,0.0,2014-10-24 12:00:33,2014-10-31 12:00:33,12,102
1101525,2278059,2090564,1.0,220.0,75.8,0.0,2012-11-01 00:00:24,6,1.0,0.0,...,NaN,1338,1338,53.974520,1.0,0.0,2014-10-28 17:15:32,2014-11-04 17:15:32,4,53
879144,2020196,1870788,1.0,199.0,66.9,0.0,2012-11-01 00:01:02,6,1.0,1.0,...,NaN,1313,1313,52.850276,1.0,0.0,2014-10-28 17:16:22,2014-11-04 17:16:22,17,61


### Extract last action and summary one

In [92]:
def data_mean_sum(column_name,target_data, raw_data):
    
    target_data[[column_name+'_mean', column_name+'_sum']] = \
    raw_data.groupby('customer_id')[column_name].agg(['mean','sum'])

In [93]:
# each customer last action is very important, like for goods and order 

def preprocessing(raw, train ="train"):
    '''
    return aggregate result of each customer
    '''
    data = pd.DataFrame(
        raw.groupby("customer_id")["customer_gender"].last().fillna(0)
    )

# columns need to calculate the min, max, mean and std
    cols = ['goods_price','order_total_payment']
    for col in cols:
        data[[col+'_min', col+'_max', col+'_mean', col+'_std']] =\
        raw.groupby('customer_id')[col].agg(['min','max','mean',np.std])
    
# columns need to calculate the count
    cols = ['order_id','goods_id' ]
    for col in cols:
        data[col+'_count'] = raw.groupby('customer_id')[col].count()

# columns need to consider about the last record
    cols =['goods_id','goods_status','goods_price','goods_has_discount',
           'goods_list_time', 'goods_delist_time','order_total_num','order_amount',
           'order_total_payment', 'order_total_discount','order_pay_time',
           'order_status', 'order_count','is_customer_rate', 'order_detail_status',
           'order_detail_goods_num','order_detail_amount','order_detail_payment',
           'order_detail_discount','customer_province_encoded','customer_city_encoded','member_status', 'is_member_actived']
    for col in cols:
        data[col+'_last'] = raw.groupby('customer_id')[col].last()

# columns need to calculate avg and sum

    cols =['is_customer_rate','order_total_payment','order_total_discount','order_count']
    for col in cols:
        data_mean_sum(col,data, train_raw)
#         data[[col+'_mean', col+'_sum']] = \
#         raw.groupby('customer_id')[col].agg(['mean','sum'])
        
        
# columns need to calculate the time diff between earliest and lastest
    cols = ['goods_list_time']
    for col in cols:
        data[col+'_interval'] = train_raw.groupby('customer_id')[col].agg([np.ptp])

    return data

In [94]:
train_raw2 = preprocessing(train_raw)

# continue to do data work on train_raw2
# columns need to calculate time character
cols = ['order_pay_time_last']

for col in cols:
    train_raw2[col +'_month'] = train_raw2[col].dt.month
    train_raw2[col +'_day'] = train_raw2[col].dt.day
    train_raw2[col +'_hour'] = train_raw2[col].dt.hour
    train_raw2[col +'_dayofweek'] = train_raw2[col].dt.dayofweek
    
train_raw2['goods_list_time_interval_day']=train_raw2['goods_list_time_interval'].dt.days
train_raw2['label'] = train_raw2.index.map(lambda x: int(x in label_raw))
# train_raw2.reset_index(inplace = True)
drop_cols = ['goods_list_time_interval','goods_list_time_last', 'goods_delist_time_last', 'order_pay_time_last']
train_raw2.drop(drop_cols, axis=1,inplace = True)


In [160]:

test = preprocessing(df)

# continue to do data work on train_raw2
# columns need to calculate time character
cols = ['order_pay_time_last']

for col in cols:
    test[col +'_month'] = test[col].dt.month
    test[col +'_day'] = test[col].dt.day
    test[col +'_hour'] = test[col].dt.hour
    test[col +'_dayofweek'] = test[col].dt.dayofweek
    
test['goods_list_time_interval_day']=test['goods_list_time_interval'].dt.days

# train_raw2.reset_index(inplace = True)
drop_cols = ['goods_list_time_interval','goods_list_time_last', 'goods_delist_time_last', 'order_pay_time_last']
test.drop(drop_cols, axis=1,inplace = True)

In [129]:
print(train_raw2.shape)
train_raw2.head()

(1435404, 44)


,customer_gender,goods_price_min,goods_price_max,goods_price_mean,goods_price_std,order_total_payment_min,order_total_payment_max,order_total_payment_mean,order_total_payment_std,order_id_count,...,order_total_discount_mean,order_total_discount_sum,order_count_mean,order_count_sum,order_pay_time_last_month,order_pay_time_last_day,order_pay_time_last_hour,order_pay_time_last_dayofweek,goods_list_time_interval_day,label
customer_id,,,,,,,,,,,,,,,,,,,,,
1000000,0.0,54.909289,54.909289,54.909289,NaN,96.9,96.9,96.90,NaN,1,...,0.0,0.0,1.0,1.0,11,1,0,3,0,0
1000034,0.0,41.310164,54.776450,48.043307,9.522102,99.9,99.9,99.90,0.000000,2,...,0.0,0.0,2.0,4.0,3,8,10,4,287,0
1000046,0.0,44.150281,44.150281,44.150281,NaN,39.9,39.9,39.90,NaN,1,...,0.0,0.0,1.0,1.0,6,7,12,4,0,0
1000048,0.0,54.866957,54.866957,54.866957,NaN,89.9,89.9,89.90,NaN,1,...,0.0,0.0,1.0,1.0,12,12,0,2,0,0
1000069,0.0,46.257409,52.996096,48.962403,3.631511,49.9,99.8,79.84,27.331356,5,...,0.0,0.0,2.6,13.0,7,13,23,5,2,0


In [161]:
print(test.shape)
test.head()

(1585986, 43)


,customer_gender,goods_price_min,goods_price_max,goods_price_mean,goods_price_std,order_total_payment_min,order_total_payment_max,order_total_payment_mean,order_total_payment_std,order_id_count,...,order_total_payment_sum,order_total_discount_mean,order_total_discount_sum,order_count_mean,order_count_sum,order_pay_time_last_month,order_pay_time_last_day,order_pay_time_last_hour,order_pay_time_last_dayofweek,goods_list_time_interval_day
customer_id,,,,,,,,,,,,,,,,,,,,,
1000000,0.0,54.909289,54.909289,54.909289,NaN,96.9,96.9,96.9,NaN,1,...,96.9,0.0,0.0,1.0,1.0,11,1,0,3,0.0
1000014,0.0,49.554744,49.554744,49.554744,NaN,66.9,66.9,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,8,3,21,5,NaN
1000034,0.0,41.310164,54.776450,48.043307,9.522102,99.9,99.9,99.9,0.0,2,...,199.8,0.0,0.0,2.0,4.0,3,8,10,4,287.0
1000046,0.0,44.150281,44.150281,44.150281,NaN,39.9,39.9,39.9,NaN,1,...,39.9,0.0,0.0,1.0,1.0,6,7,12,4,0.0
1000048,0.0,54.866957,54.866957,54.866957,NaN,89.9,89.9,89.9,NaN,1,...,89.9,0.0,0.0,1.0,1.0,12,12,0,2,0.0


In [130]:
# save results to pickle files
train_raw2.to_pickle(path+'processed_train_data.pkl')

In [162]:
test.to_pickle(path+'processed_test_data.pkl')

## Create Baseline Model _lightgbm

In [105]:
#  150582 out of 173385. 87% August purchaser does not have historical records
print(len(set(label_raw)))
print(len(set(label_raw).intersection(train_raw.customer_id)))
len(set(label_raw)- set(train_raw.customer_id))


173385
22803


150582

In [139]:
X_train, X_test, y_train, y_test= train_test_split(train_raw2.drop(['label'],axis=1),\
                                                   train_raw2['label'],
                                                  test_size =0.2, random_state =43)

In [140]:
clf= lgb.LGBMClassifier(
num_leaves = 2**5 -1, reg_alpha =0.25, reg_lambda =0.25, objective ='binary',
    max_depthe =5, learning_rate = 0.005, min_child_samples =3,
    random_state =2021, n_estimators =2000, subsample =1, colsample_bytree = 1
)

clf.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=1, learning_rate=0.005, max_depthe=5,
               min_child_samples=3, n_estimators=2000, objective='binary',
               random_state=2021, reg_alpha=0.25, reg_lambda=0.25, subsample=1)

In [142]:
y_pred =clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9355861237769131

In [145]:
y_pred = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_pred)

0.6435000204202034

In [148]:
# check the importance of the model
pd.DataFrame({'column': X_train.columns,
                 'importance': clf.feature_importances_}).sort_values(by='importance',ascending= False)

,column,importance
28,customer_city_encoded_last,4681
4,goods_price_std,3804
39,order_pay_time_last_day,3688
40,order_pay_time_last_hour,3199
27,customer_province_encoded_last,3109
2,goods_price_max,2928
11,goods_id_last,2607
1,goods_price_min,2438
38,order_pay_time_last_month,2402
26,order_detail_discount_last,2131


In [163]:
clf.predict(test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [44]:
train_raw[train_raw.customer_id == 1000034][['goods_id','goods_status','goods_price','goods_has_discount',\
          'goods_list_time','goods_delist_time','order_total_num','order_amount', 'order_total_payment', 'order_total_discount',\
         'order_pay_time', 'order_status', 'order_count','is_customer_rate',\
         'order_detail_status','order_detail_goods_num','order_detail_amount',\
         'order_detail_payment','order_detail_discount']]

,goods_id,goods_status,goods_price,goods_has_discount,goods_list_time,goods_delist_time,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,order_detail_status,order_detail_goods_num,order_detail_amount,order_detail_payment,order_detail_discount
157749,2853,2.0,41.310164,0.0,2014-01-10 15:54:23,2014-01-11 12:46:14,2.0,299.8,99.9,0.0,2013-03-08 10:21:29,6,2.0,0.0,6.0,1.0,0.0,0.0,59.9
157750,998,1.0,54.776450,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07,2.0,299.8,99.9,0.0,2013-03-08 10:21:29,6,2.0,0.0,6.0,1.0,99.9,99.9,140.0
